# CLIP

Revisión informal de la arquitectura _CLIP_ (_Contrastive Language-Image Pre-Training_).

Modelo multimodal (lenguaje y visión) de código abierto publicado por OpenAI en enero de 2021.

- [https://arxiv.org/pdf/2103.00020](https://arxiv.org/pdf/2103.00020)

Aunque los términos arquitectura y modelo se utilicen indistintamente, es habitual presentar _CLIP_ como un método de _pre-entrenamiento_.

## 1. Arquitectura

El modelo utiliza dos _encoders_, uno para textos y otro para imágenes, cuyas salidas se comparan para medir su similitud.

```
  ┌────> Text Encoder ─────┐
  |                        ↓
Input                    Output
  |                        ↑
  └────> Image Encoder ────┘
```

La idea general es que ambos _encoders_ proyectan sus entradas a vectores dentro de un espacio de características alineado, lo que permite tratarlos de manera homogénea.

### 1.1. Funcionamiento General

El modelo recibe dos tipos de entrada: textos e imágenes.

#### 1.1.1 Textos

```
Text ─> [SOT] Tokens [EOT] -> Embeddings -> Text Encoder -> Features
```

- Los textos se convierten en secuencias de _tokens_
- Se añaden _tokens_ especiales de inicio _SOT_ y final _EOT_ de texto a las secuencias
- Los _tokens_ se convierten en _embeddings_
- Los _embeddings_ se procesan por el _encoder_ de texto para enriquecerlos con información de contexto aplicando el mecanismo de atención
- Se toma el resultado de la salida del _encoder_ correspondiente al _token_ _EOT_
- Y se calcula el vector de características (_features_) proyectando el resultado obtenido

#### 1.1.2. Imágenes

```
Image ─> Patches ─> [CLS] Embeddings -> Image Encoder -> Features
```

- Las imágenes se dividen en cuadrículas (_patches_)
- Los _patches_ se convierten en secuencias de _embeddings_
- Se añade un _embedding_ especial _CLS_ al inicio de las secuencias
- Los _embeddings_ se procesan por el _encoder_ de imágenes para enriquecerlos con información de contexto aplicando el mecanismo de atención
- Se toma el resultado de la salida del _encoder_ correspondiente al _embedding_ _CLS_
- Y se calcula el vector de características (_features_) proyectando el resultado obtenido

#### 1.1.3. Similitud

```
Text ─> [SOT] Tokens [EOT] -> Embeddings -> Text Encoder -> Features ─┐
                                                                      ↓
                                                                  Similarity
                                                                      ↑
Image ─> Patches ─> [CLS] Embeddings -> Image Encoder -> Features ────┘
```

La salida del modelo es el resultado de aplicar la medida de _similitud coseno_ que mide la semejanza existente entre dos vectores.

### 1.2. Ejemplo

Dada una lista de textos (clases):
- $ \text{Un libro en una mesa} $
- $ \text{La torre Eiffel} $
- $ \text{Un grupo de personas} $

Y una imagen:
- $ \text{paris.png} $

La salida del modelo puede utilizarse para obtener un vector con las probabilidades para cada clase:
- $ \begin{bmatrix} 0.1 & 0.7 & 0.2 \end{bmatrix} $

Este es un ejemplo de clasificación conocido como _zero-shot_, donde el modelo puede clasificar imágenes en clases con las que no fue explícitamente entrenado.

## 3. Text Encoder

El _encoder_ de texto utiliza una arquitectura _Transformer_ en la que, en vez de proyectar la salida de la atención a una matriz de _scores_ con el tamaño del vocabulario, proyecta la salida de la atención correspondiente al _token_ _EOT_ a un vector de características.

```
Text ─> [SOT] Tokens [EOT] ─> Embeddings ─> PE ─> N x Transformer ─> Norm ─> Features
```

Algunas líneas relevantes del código fuente:

```python
x = self.token_embedding(text).type(self.dtype)
x = x + self.positional_embedding.type(self.dtype)
x = self.transformer(x)
x = self.ln_final(x).type(self.dtype)

# take features from the eot embedding (eot_token is the highest number in each sequence)
x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ self.text_projection
```

### 3.1. Tokenizer

Cada texto de entrada se procesa por un _tokenizer_ que convierte texto libre de tamaño arbitrario en una secuencia de _tokens_ $T$.

El _tokenizer_ final entrenado tiene un vocabulario de $V$ entradas, incluyendo los _tokens_ especiales de inicio (_SOT_) y final de texto (_EOT_), que se añaden a las secuencias de _tokens_ antes de introducirlas en el modelo.

Las secuencias generadas se truncan a una longitud máxima prefijada de $n$ _tokens_ como tamaño máximo de la ventana de contexto, completando con ceros las secuencias de menor longitud a modo de _padding_.

Dimensiones:

- $ T \in \{0, 1, \dots, V-1\}^{1 \times n} $

Algunas líneas relevantes del código fuente:

```python
sot_token = _tokenizer.encoder["<|startoftext|>"]
eot_token = _tokenizer.encoder["<|endoftext|>"]

all_tokens = [[sot_token] + _tokenizer.encode(text) + [eot_token] for text in texts]

torch.zeros(len(all_tokens), context_length, ...)
```

### 3.2. Embedding

La primera capa del _encoder_ convierte los _tokens_ en _embeddings_.

Su matriz de pesos $W$ tiene tantas filas $V$ como entradas tiene el vocabulario, y tantas columnas como dimensiones $d$ se deseen para los _embeddings_.

- $ W = \begin{bmatrix}
w_{1,1} & w_{1,2} & \ldots & w_{1,d}
\\ w_{2,1} & w_{2,2} & \ldots & w_{2,d}
\\ \vdots & \vdots & \ddots & \vdots
\\ w_{V,1} & w_{V,2} & \ldots & w_{V,d}
\end{bmatrix} $

La matriz es inicializada con una distribución normal, y aprendida durante el entrenamiento.

Los _embeddings_ se obtienen al indexar la matriz de pesos $W$ con cada _token_ de la secuencia $T$.

- $ X = \begin{bmatrix}
W[T_1]
\\ W[T_2]
\\ \vdots
\\ W[T_n]
\end{bmatrix} = \begin{bmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,d}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,d}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{n,1} & x_{n,2} & \ldots & x_{n,d}
\end{bmatrix} $

La secuencia de _embeddings_ $X$ se propaga a través del _encoder_ sometiéndose a transformaciones funcionales en cada capa.

Dimensiones:

- $ T \in \{0, 1, \dots, V-1\}^{1 \times n} $

- $ W \in \mathbb{R}^{V \times d} $

- $ X \in \mathbb{R}^{n \times d} $

Huelga decir que el modelo funciona con _batches_, por lo que las dimensiones reales son (_tamaño de batch_, _longitud de secuencia_, _dimensiones de los embeddings_), pero por simplicidad en el análisis se omite.

Algunas líneas relevantes del código fuente:

```python
self.token_embedding = nn.Embedding(vocab_size, transformer_width)

nn.init.normal_(self.token_embedding.weight, std=0.02)

x = self.token_embedding(text).type(self.dtype) 
```

### 3.3. Positional Embedding

Esta capa enriquece los _embeddings_ con información posicional.

La información se añade a través de una matriz de pesos $W$, inicializada con una distribución normal, y aprendida durante el entrenamiento.

Esta forma de trabajar se diferencia de otras técnicas habituales como _Positional Encoding_ (_PE_), o _Rotary Positional Embedding_ (_RoPE_), que aplican una función para calcular la información a añadir.

La matriz de pesos tiene tantas filas $n$ como _tokens_ admite la ventana máxima de contexto, y tantas columnas como dimensiones $d$ tienen los _embeddings_.

- $ W = \begin{bmatrix}
w_{1,1} & w_{1,2} & \ldots & w_{1,d}
\\ w_{2,1} & w_{2,2} & \ldots & w_{2,d}
\\ \vdots & \vdots & \ddots & \vdots
\\ w_{n,1} & w_{n,2} & \ldots & w_{n,d}
\end{bmatrix} $

Los _embeddings_ se enriquecen con la información posicional sumándoles la matriz de pesos $W$.

- $ PE = X + W = \begin{bmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,d}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,d}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{n,1} & x_{n,2} & \ldots & x_{n,d}
\end{bmatrix} + \begin{bmatrix}
w_{1,1} & w_{1,2} & \ldots & w_{1,d}
\\ w_{2,1} & w_{2,2} & \ldots & w_{2,d}
\\ \vdots & \vdots & \ddots & \vdots
\\ w_{n,1} & w_{n,2} & \ldots & w_{n,d}
\end{bmatrix} $

La salida de este paso son los _embeddings_ enriquecidos con la información posicional.

- $ X \leftarrow PE $

Dimensiones:

- $ X \in \mathbb{R}^{n \times d} $

- $ W \in \mathbb{R}^{n \times d} $

- $ PE \in \mathbb{R}^{n \times d} $

Algunas líneas relevantes del código fuente:

```python
self.positional_embedding = nn.Parameter(torch.empty(self.context_length, transformer_width))

nn.init.normal_(self.positional_embedding, std=0.01)

x = x + self.positional_embedding.type(self.dtype)
```

### 3.4. Transformer

Esta capa añade información de contexto a los _embeddings_, ya enriquecidos con información posicional, haciéndolos pasar por $N$ _transformers_.

Cada _transformer_ se compone de una capa de atención y otra de _feed forward_, con capas intermedias de normalización y conexión residual.

```
N x (─> Norm ─> MHA ─> Add ─> Norm ─> FF ─> Add)
     │                  ↑  │                 ↑
     └──────────────────┘  └─────────────────┘
```

La salida del primer _transformer_ alimenta la entrada del segundo _transformer_, la salida del segundo la entrada del tercero, y así sucesivamente.

Algunas líneas relevantes del código fuente:

```python
self.ln_1 = LayerNorm(d_model)
self.attn = nn.MultiheadAttention(d_model, n_head)
self.ln_2 = LayerNorm(d_model)
self.mlp = nn.Sequential(OrderedDict([
  ("c_fc", nn.Linear(d_model, d_model * 4)),
  ("gelu", QuickGELU()),
  ("c_proj", nn.Linear(d_model * 4, d_model))
]))

x = x + self.attention(self.ln_1(x))
x = x + self.mlp(self.ln_2(x))
```

#### 3.4.1. Norm (Pre-Multi-Head Attention)

Esta capa normaliza los _embeddings_ utilizando _Layer Normalization_.

- [https://arxiv.org/pdf/1607.06450](https://arxiv.org/pdf/1607.06450)

Esta función de normalización ajusta los valores de entrada restando la media y dividiendo por la raíz cuadrada de la varianza, más una constante $\epsilon$ para evitar divisiones por cero, y aplicando una transformación afín con dos parámetros $\gamma$ y $\beta$ aprendidos durante el entrenamiento.

- $ \operatorname{LayerNorm}(x) = \cfrac{x - \mu[x]}{\sqrt{\sigma[x] + \epsilon}} \gamma + \beta $

Para el primer _transformer_, la función se aplica, fila a fila, a los _embeddings_ enriquecidos con información posicional.

- $ X \leftarrow \begin{bmatrix}
\operatorname{LayerNorm}(X_1)
\\ \operatorname{LayerNorm}(X_2)
\\ \vdots
\\ \operatorname{LayerNorm}(X_n)
\end{bmatrix} $

Y para el resto de _transformers_, se calcula aplicando la función a la salida del _transformer_ anterior.

Dimensiones:

- $ X \in \mathbb{R}^{n \times d} $

Algunas líneas relevantes del código fuente:

```python
self.ln_1 = LayerNorm(d_model)

self.ln_1(x)
```

#### 3.4.2. Multi-Head Attention (MHA)

Esta capa implementa el mecanismo de atención según se describe en el _paper_ seminal "_Attention Is All You Need_".

- [https://arxiv.org/pdf/1706.03762](https://arxiv.org/pdf/1706.03762)

Esta capa tiene $H$ cabezas de atención con sus correspondientes matrices de pesos $W_Q^h$, $W_K^h$ y $W_V^h$, aprendidas durante el entrenamiento.

- $ d_k = d_v = \cfrac{d}{H} $

- $ W_Q^h \in \mathbb{R}^{d \times d_k} $
 
- $ W_K^h \in \mathbb{R}^{d \times d_k} $
  
- $ W_V^h \in \mathbb{R}^{d \times d_v} $

Los _embeddings_ normalizados se proyectan por cada una de las matrices de pesos para obtener las matrices $Q^h$ (_Query_), $K^h$ (_Key_) y $V^h$ (_Value_).

- $ Q^h = X W_Q^h $

- $ K^h = X W_K^h $

- $ V^h = X W_V^h $

Y las proyecciones obtenidas se utilizan para calcular una matriz cuadrada de $n \times n$ dimensiones.

- $ S^h = \operatorname{softmax}\left(\cfrac{Q^h {K^h}^T}{\sqrt{d_k}}\right) $

Las matrices de pesos proyectan los _embeddings_ de entrada a tres espacios de características distintos.

El producto escalar entre dos de las proyecciones compara dos de los espacios de características, valores altos indican alta similaridad.

El resultado del producto se escala, y se aplica la función $softmax$ para convertir el resultado en una distribución de probabilidad.

De forma que la matriz cuadrada resultante indica para cada _embedding_ que atención presta a si mismo y al resto de _embeddings_ de la secuencia.

La matriz $V^h$ en el tercer espacio de características se pondera por la distribución de probabilidad para obtener las salidas de las cabezas de atención.

- $ A^h = S^h V^h $

Y finalmente, se concatenan todas las matrices de atención, y se proyectan por una matriz de pesos $W_O$ que retorna el resultado a las dimensiones de los _embeddings_.

- $ X \leftarrow \operatorname{concat}(A^1, A^2 \ldots, A^H) W_O $

Aunque no se indica de manera explícita, se utilizan máscaras, para ignorar _tokens_ de _padding_, y para no calcular la atención de un _token_ respecto a _tokens_ posteriores aún no emitidos.

Dimensiones:

- $ X \in \mathbb{R}^{n \times d} $

- $ Q^h \in \mathbb{R}^{n \times d_k} $

- $ K^h \in \mathbb{R}^{n \times d_k} $

- $ V^h \in \mathbb{R}^{n \times d_v} $

- $ A^h \in \mathbb{R}^{n \times d_v} $

- $ W_O \in \mathbb{R}^{H d_v \times d} $

Algunas líneas relevantes del código fuente:

```python
self.attn = nn.MultiheadAttention(d_model, n_head)

self.attention(self.ln_1(x))
```

#### 3.4.3. Add (Post-Multi-Head Attention)

Esta capa realiza una conexión residual para propagar información originalmente presente en la entrada del _transformer_.

Para el primer _transformer_, se suma a la salida de la atención la entrada al _transformer_, que son los _embeddings_ enriquecidos con información posicional.

- $ ATN = X + PE $

Y para el resto de _transformers_, se suma a la salida de la atención del _transformer_ en curso la salida del _transformer_ anterior.

La salida de este paso es el resultado de la suma.

- $ X \leftarrow ATN $

Dimensiones:

- $ X \in \mathbb{R}^{n \times d} $

- $ P \in \mathbb{R}^{n \times d} $

- $ A \in \mathbb{R}^{n \times d} $

Algunas líneas relevantes del código fuente:

```python
x = x + self.attention(self.ln_1(x))
```

#### 3.4.4. Norm (Pre-Feed Forward)

Esta capa normaliza los _embeddings_ utilizando _Layer Normalization_.

- $ X \leftarrow \begin{bmatrix}
\operatorname{LayerNorm}(X_1)
\\ \operatorname{LayerNorm}(X_2)
\\ \vdots
\\ \operatorname{LayerNorm}(X_n)
\end{bmatrix} $

Dimensiones:

- $ X \in \mathbb{R}^{n \times d} $

Algunas líneas relevantes del código fuente:

```python
self.ln_2 = LayerNorm(d_model)

self.ln_2(x)
```

#### 3.4.5. Feed Forward (FF)

Esta capa añade no linealidad expandiendo las dimensiones internas del modelo para mejorar la potencia expresiva del mismo.

Aplica una primera proyección lineal que expande cuatro veces las dimensiones del modelo.

Aplica la función _GELU_ (_Gaussian Error Linear Units_) para añadir no linealidad.

- [https://arxiv.org/pdf/1606.08415](https://arxiv.org/pdf/1606.08415)

Y aplica una segunda proyección lineal para reducir el modelo a sus dimensiones originales.

- $ X \leftarrow \operatorname{GELU}(X W_1) W_2 $

Ambas matrices de pesos son aprendidas durante el entrenamiento.

Dimensiones:

- $ X \in \mathbb{R}^{n \times d} $

- $ W_1 \in \mathbb{R}^{d \times 4d} $

- $ W_2 \in \mathbb{R}^{4d \times d} $

Algunas líneas relevantes del código fuente:

```python
self.mlp = nn.Sequential(OrderedDict([
  ("c_fc", nn.Linear(d_model, d_model * 4)),
  ("gelu", QuickGELU()),
  ("c_proj", nn.Linear(d_model * 4, d_model))
]))

self.mlp(self.ln_2(x))
```

#### 3.4.6. Add (Post-Feed Forward)

Esta capa realiza una conexión residual para propagar información originalmente presente antes del _feed forward_.

Suma a la salida de la capa anterior la salida de la conexión residual aplicada tras la atención.

- $ X \leftarrow X + ATN $

Dimensiones:

- $ X \in \mathbb{R}^{n \times d} $

- $ A \in \mathbb{R}^{n \times d} $

Algunas líneas relevantes del código fuente:

```python
x = x + self.mlp(self.ln_2(x))
```

### 3.5. Norm (Final)

Esta capa normaliza los _embeddings_ utilizando _Layer Normalization_.

- $ X \leftarrow \begin{bmatrix}
\operatorname{LayerNorm}(X_1)
\\ \operatorname{LayerNorm}(X_2)
\\ \vdots
\\ \operatorname{LayerNorm}(X_n)
\end{bmatrix} $

Dimensiones:

- $ X \in \mathbb{R}^{n \times d} $

Algunas líneas relevantes del código fuente:

```python
self.ln_final = LayerNorm(transformer_width)

x = self.ln_final(x).type(self.dtype)
```

### 3.6. Features

Esta capa calcula la salida final del _encoder_ de texto.

Proyecta un _embedding_ específico a un espacio de caracteristicas de $m$ dimensiones mediante una matriz $W$, inicializada con una distribución normal, y aprendida durante el entrenamiento.

- $ W = \begin{bmatrix}
w_{1,1} & w_{1,2} & \ldots & w_{1,m}
\\ w_{2,1} & w_{2,2} & \ldots & w_{2,m}
\\ \vdots & \vdots & \ddots & \vdots
\\ w_{d,1} & w_{d,2} & \ldots & w_{d,m}
\end{bmatrix} $

La capa localiza la posición $k$ del _token_ _EOT_ dentro de la secuencia original $T$ de _tokens_ de entrada.

- $ k = \operatorname{lookup}(\text{EOT}, T) $

Toma el _embedding_ correspondiente a dicha posición de la secuencia de _embeddings_.

- $ Y = X[k] $

Y lo proyecta mediante la matriz de pesos $W$ para obtener el vector de caracteristicas $F$ de $m$ dimensiones.

- $ F = Y W = \begin{bmatrix}
y_1 & y_2 & \ldots & y_d
\end{bmatrix} \begin{bmatrix}
w_{1,1} & w_{1,2} & \ldots & w_{1,m}
\\ w_{2,1} & w_{2,2} & \ldots & w_{2,m}
\\ \vdots & \vdots & \ddots & \vdots
\\ w_{d,1} & w_{d,2} & \ldots & w_{d,m}
\end{bmatrix} = \begin{bmatrix}
f_1 & f_2 & \ldots & f_m
\end{bmatrix}$

Dimensiones:

- $ T \in \{0, 1, \dots, V-1\}^{1 \times n} $

- $ X \in \mathbb{R}^{n \times d} $

- $ Y \in \mathbb{R}^{1 \times d} $

- $ W \in \mathbb{R}^{d \times m} $

- $ F \in \mathbb{R}^{1 \times m} $

Algunas líneas relevantes del código fuente:

```python
self.text_projection = nn.Parameter(torch.empty(transformer_width, embed_dim))

nn.init.normal_(self.text_projection, std=self.transformer.width ** -0.5)

# take features from the eot embedding (eot_token is the highest number in each sequence)
x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ self.text_projection
```

## 4. Image Encoder

El _encoder_ de imágenes utiliza una arquitectura _Transformer_ en la que, en vez de proyectar la salida a una matriz de _scores_ con el tamaño del vocabulario, proyecta la salida correspondiente a un _emebedding_ especial _CLS_ a un vector de características.

```
Image ─> Patches ─> [CLS] Embeddings ─> PE ─> Norm ─> M x Transformer ─> Norm ─> Features
```

Algunas líneas relevantes del código fuente:

```python
x = self.conv1(x)
x = torch.cat([self.class_embedding.to(x.dtype) + torch.zeros(x.shape[0], 1, x.shape[-1]), x], dim=1)
x = x + self.positional_embedding.to(x.dtype)
x = self.ln_pre(x)
x = self.transformer(x)
x = self.ln_post(x[:, 0, :])

x = x @ self.proj
```

Aclarar que el _paper_ de _CLIP_ propone dos arquitecturas para el _encoder_ de imágenes. Una basada en una arquitectura llamada _ResNet_. Y otra basada en _Vision Transformer_ (_ViT_), que es la que analiza aquí.

### 4.1. Embeddings

De igual forma que el texto libre se convierte en una secuencia de _tokens_ con un _tokenizer_, las imágenes se dividen en una secuencia de cuadriculas (_patches_) con una capa convolucional.

Cada imagen de entrada $I$ se trata como un tensor de tamaño fijo $W \times H$ con tres componentes de color (_RGB_).

El número $p$ de _patches_ a generar depende de las dimensiones de la imagen y el tamaño $S \times S$ de los _patches_.

- $ p = \cfrac{W}{S} \times \cfrac{H}{S} $

El tamaño de _kernel_ y _stride_ de la capa convolucional se configuran con el mismo valor $S$ para dividir la imagen en cuadriculas que no se solapan entre sí y sin dejar espacios intermedios. 

Y la dimensión de salida de la capa convolucional se configura con un valor $d$ para que genere un vector de $d$ dimensiones por cada _patch_.

Los $p$ vectores generados por la capa son el resultado de aplicar $d$ matrices de pesos (_kernels_), aprendidas durante el entrenamiento, a cada _patch_ de tamaño $3 \times S \times S$.

Los vectores de $d$ dimensiones generados por la capa se interpretan directamente como _embeddings_. De forma que la salida de este paso es la secuencia de $p$ _embeddings_ generados a partir de la imagen $I$ de entrada.

- $ X = \begin{bmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,d}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,d}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{p,1} & x_{p,2} & \ldots & x_{p,d}
\end{bmatrix} $

La secuencia de _embeddings_ $X$ se propaga a través del _encoder_ sometiéndose a transformaciones funcionales en cada capa.

Dimensiones:

- $ I \in \mathbb{R}^{3 \times W \times H} $

- $ X \in \mathbb{R}^{p \times d} $

Huelga decir que el modelo funciona con _batches_, por lo que las dimensiones reales son (_tamaño de batch_, _longitud de secuencia_, _dimensiones de los embeddings_), pero por simplicidad en el análisis se omite.

Algunas líneas relevantes del código fuente:

```python
self.conv1 = nn.Conv2d(in_channels=3, out_channels=width, kernel_size=patch_size, stride=patch_size, bias=False)

x = self.conv1(x)  # shape = [*, width, grid, grid]
x = x.reshape(x.shape[0], x.shape[1], -1)  # shape = [*, width, grid ** 2]
x = x.permute(0, 2, 1)  # shape = [*, grid ** 2, width]
```

### 4.2. Class Embedding

A la secuencia de _embeddings_ obtenida en la capa anterior se le antepone un _embedding_ especial _CLS_ (_Class Embedding_).

Este _embedding_ es un vector de $d$ dimensiones aprendido por el modelo, no un valor fijo predefinido como los _tokens_ _SOT_ y _EOT_ utilizados en el _encoder_ de texto.

- $ \text{CLS} = \begin{bmatrix}
\text{cls}_1 & \text{cls}_2 & \ldots & \text{cls}_d
\end{bmatrix} $

Se inicializa con valores tomados de una distribución normal, y se actualiza durante el entrenamiento.

La salida de este paso es la secuencia de $n = p + 1$ _embeddings_ resultante de añadir el _embedding_ especial _CLS_ al inicio de la secuencia $X$.

- $ X \leftarrow \begin{bmatrix}
\text{cls}_1 & \text{cls}_2 & \ldots & \text{cls}_d
\\ x_{1,1} & x_{1,2} & \ldots & x_{1,d}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,d}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{p,1} & x_{p,2} & \ldots & x_{p,d}
\end{bmatrix} $

Dimensiones:

- $ X_{in} \in \mathbb{R}^{p \times d} $

- $ \text{CLS} \in \mathbb{R}^{1 \times d} $

- $ X_{out} \in \mathbb{R}^{n \times d} $

Algunas líneas relevantes del código fuente:

```python
scale = width ** -0.5
self.class_embedding = nn.Parameter(scale * torch.randn(width))

x = torch.cat([self.class_embedding.to(x.dtype) + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device), x], dim=1)  # shape = [*, grid ** 2 + 1, width]
```

### 4.3. Positional Embedding

Esta capa enriquece los _embeddings_ con información posicional de igual forma que se hace en el _encoder_ de texto.

La información se añade a través de una matriz de pesos $W$, inicializada con una distribución normal, y aprendida durante el entrenamiento.

- $ W = \begin{bmatrix}
w_{1,1} & w_{1,2} & \ldots & w_{1,d}
\\ w_{2,1} & w_{2,2} & \ldots & w_{2,d}
\\ \vdots & \vdots & \ddots & \vdots
\\ w_{n,1} & w_{n,2} & \ldots & w_{n,d}
\end{bmatrix} $

La salida de esta capa se obtiene sumando a la secuencia de _embeddings_ la matriz de pesos $W$.

- $ X \leftarrow X + W = \begin{bmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,d}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,d}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{n,1} & x_{n,2} & \ldots & x_{n,d}
\end{bmatrix} + \begin{bmatrix}
w_{1,1} & w_{1,2} & \ldots & w_{1,d}
\\ w_{2,1} & w_{2,2} & \ldots & w_{2,d}
\\ \vdots & \vdots & \ddots & \vdots
\\ w_{n,1} & w_{n,2} & \ldots & w_{n,d}
\end{bmatrix} $

Dimensiones:

- $ X \in \mathbb{R}^{n \times d} $

- $ W \in \mathbb{R}^{n \times d} $

Algunas líneas relevantes del código fuente:

```python
scale = width ** -0.5
self.positional_embedding = nn.Parameter(scale * torch.randn((input_resolution // patch_size) ** 2 + 1, width))

x = x + self.positional_embedding.to(x.dtype)
```

### 4.4. Norm (Pre-Transformer)

Esta capa normaliza los _embeddings_ utilizando _Layer Normalization_ de igual forma que otras capas de normalización del modelo.

- $ X \leftarrow \begin{bmatrix}
\operatorname{LayerNorm}(X_1)
\\ \operatorname{LayerNorm}(X_2)
\\ \vdots
\\ \operatorname{LayerNorm}(X_n)
\end{bmatrix} $

Dimensiones:

- $ X \in \mathbb{R}^{n \times d} $

Algunas líneas relevantes del código fuente:

```python
self.ln_pre = LayerNorm(width)

x = self.ln_pre(x)
```

### 4.5. Transformer

Esta capa añade información de contexto a los _embeddings_, ya enriquecidos con información posicional, haciéndolos pasar por $M$ _transformers_.

Cada _transformer_ se compone de una capa de atención y otra de _feed forward_, con capas intermedias de normalización y conexión residual.

```
M x (─> Norm ─> MHA ─> Add ─> Norm ─> FF ─> Add)
     │                  ↑  │                 ↑
     └──────────────────┘  └─────────────────┘
```

La salida del primer _transformer_ alimenta la entrada del segundo _transformer_, la salida del segundo la entrada del tercero, y así sucesivamente.

Sigue el mismo proceso que el descrito para el _encoder_ de texto, enriqueciendo los _embeddings_ $X$ con información de contexto.

La salida de esta capa es la salida final del último _transformer_.

Dimensiones:

- $ X \in \mathbb{R}^{n \times d} $

Algunas líneas relevantes del código fuente:

```python
self.ln_1 = LayerNorm(d_model)
self.attn = nn.MultiheadAttention(d_model, n_head)
self.ln_2 = LayerNorm(d_model)
self.mlp = nn.Sequential(OrderedDict([
  ("c_fc", nn.Linear(d_model, d_model * 4)),
  ("gelu", QuickGELU()),
  ("c_proj", nn.Linear(d_model * 4, d_model))
]))

x = x + self.attention(self.ln_1(x))
x = x + self.mlp(self.ln_2(x))
```

### 4.6. Norm (Post-Transformer)

Esta capa normaliza los _embeddings_ aplicando _Layer Normalization_, pero sólo sobre el primer elemento, que se corresponde con el _embedding_ especial _CLS_.

- $ Y = \operatorname{LayerNorm}(X_1) $

Dimensiones:

- $ X \in \mathbb{R}^{n \times d} $

- $ Y \in \mathbb{R}^{1 \times d} $

Algunas líneas relevantes del código fuente:

```python
self.ln_post = LayerNorm(width)

x = self.ln_post(x[:, 0, :])
```

### 4.7. Features

Esta capa calcula la salida final del _encoder_ de imagen.

Proyecta el _embedding_ especial _CLS_ a un espacio de caracteristicas de $m$ dimensiones mediante una matriz $W$, inicializada con una distribución normal, y aprendida durante el entrenamiento.

- $ W = \begin{bmatrix}
w_{1,1} & w_{1,2} & \ldots & w_{1,m}
\\ w_{2,1} & w_{2,2} & \ldots & w_{2,m}
\\ \vdots & \vdots & \ddots & \vdots
\\ w_{d,1} & w_{d,2} & \ldots & w_{d,m}
\end{bmatrix} $

La capa proyecta el _embedding_ por la matriz $W$ para obtener el vector de caracteristicas $F$ de $m$ dimensiones.

- $ F = Y W = \begin{bmatrix}
y_1 & y_2 & \ldots & y_d
\end{bmatrix} \begin{bmatrix}
w_{1,1} & w_{1,2} & \ldots & w_{1,m}
\\ w_{2,1} & w_{2,2} & \ldots & w_{2,m}
\\ \vdots & \vdots & \ddots & \vdots
\\ w_{d,1} & w_{d,2} & \ldots & w_{d,m}
\end{bmatrix} = \begin{bmatrix}
f_1 & f_2 & \ldots & f_m
\end{bmatrix}$

Dimensiones:

- $ Y \in \mathbb{R}^{1 \times d} $

- $ W \in \mathbb{R}^{d \times m} $

- $ F \in \mathbb{R}^{1 \times m} $

Algunas líneas relevantes del código fuente:

```python
scale = width ** -0.5
self.proj = nn.Parameter(scale * torch.randn(width, output_dim))

x = x @ self.proj
```

## 5. Similarity

Los vectores de características obtenidos por los _encoders_ se comparan utilizando la _similitud coseno_ (_Cosine Similarity_).

```
Text ─> Text Encoder ─────┐
                          ↓
                 Cosine Similarity ─> Logits
                          ↑
Image ──> Image Encoder ──┘
```

La entrada de este paso son los vectores de características resultantes de aplicar los _encoders_ a las entradas del modelo.

- $ F_T = \operatorname{encode}(\text{text}) $

- $ F_I = \operatorname{encode}(\text{image}) $

Y la salida son los conocidos como "_logits_". Es decir, las puntuaciones brutas (_scores_) resultantes de comparar los dos vectores de características. Es habitual aplicar a la salida del modelo la función $softmax$ para convertir las puntuaciones en una distribución de probabilidad con valores entre 0 y 1.

Para un ejemplo de clasificación de tipo _zero-shot_, a la entrada se tiene una imagen y varios textos, y a su salida una matriz de _scores_ en vez de un único valor. Por lo que, por claridad en el cálculo de la salida del modelo en este apartado, se supone que la entrada al modelo es un _batch_ de $N$ imágenes y $M$ secuencias de _tokens_.

Dimensiones:

- $ I \in \mathbb{R}^{N \times 3 \times W \times H} $

- $ T \in \mathbb{R}^{M \times n} $

- $ F_I \in \mathbb{R}^{N \times m} $

- $ F_T \in \mathbb{R}^{M \times m} $

Algunas líneas relevantes del código fuente:

```python
self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))

image_features = self.encode_image(image)
text_features = self.encode_text(text)

image_features = image_features / image_features.norm(dim=1, keepdim=True)
text_features = text_features / text_features.norm(dim=1, keepdim=True)

logit_scale = self.logit_scale.exp()
logits_per_image = logit_scale * image_features @ text_features.t()
logits_per_text = logits_per_image.t()

# shape = [global_batch_size, global_batch_size]
return logits_per_image, logits_per_text
```

### 5.1. Cosine Similarity

La _similitud coseno_ se calcula como el producto escalar entre dos vectores unitarios.

El producto escalar es una medida de la semejanza de dos vectores, a mayor semejanza, mayor magnitud.

- $ a \cdot b = \lVert a \rVert \lVert b \rVert \cos{\theta} $

Si los vectores son unitarios entonces el producto escalar es directamente el coseno del ángulo $\theta$ que forman los dos vectores entre sí.

- $ a \cdot b = \cos{\theta} $

Y el producto escalar se calcula como la suma de los productos de los componentes de los dos vectores, uno a uno.

- $  a \cdot b = \sum\limits_{i=1}^{n} a_i b_i $

El modelo utiliza específicamente la norma _L2_ para calcular la longitud de los vectores y convertirlos en unitarios.

- $ \lVert x \rVert_2 = \sqrt{\sum\limits_{i=1}^{n} x_i^2} $

La salida de este paso es la _similitud coseno_ calculada como el producto de los vectores de características normalizados.

- $ S = \cfrac{F_I \cdot F_T}{\lVert F_I \rVert_2 \lVert F_T \rVert_2} $

Dimensiones:

- $ F_I \in \mathbb{R}^{N \times m} $

- $ F_T \in \mathbb{R}^{M \times m} $

- $ S \in \mathbb{R}^{N \times M} $

Algunas líneas relevantes del código fuente:

```python
image_features = image_features / image_features.norm(dim=1, keepdim=True)
text_features = text_features / text_features.norm(dim=1, keepdim=True)

logit_scale * image_features
```

### 5.2. Logits

Los _logits_ son las puntuaciones finales de similitud entre las representaciones de las imágenes y los textos, y viceversa.

Se obtienen a partir de la _similitud coseno_ escalada por la exponencial de un factor de _temperatura_ $\tau$ aprendido durante el entrenamiento.

- $ S_I = e^{\tau} S $

Cada elemento $(i, j)$ de esta matriz representa la similitud (_logit_) entre la imagen $i$-ésima y el texto $j-$ésimo.

Los _logits_ de los textos frente a las imágenes son la traspuesta de la matriz anterior.

- $ S_T = S_I^T $

Cada elemento $(i, j)$ de esta matriz representa la similitud (_logit_) entre el texto $i$-ésimo y la imagen $j-$ésima.

Dimensiones:

- $ S \in \mathbb{R}^{N \times M} $

- $ \tau \in \mathbb{R} $

- $ S_I \in \mathbb{R}^{N \times M} $

- $ S_T \in \mathbb{R}^{M \times N} $

Algunas líneas relevantes del código fuente:

```python
self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))

logit_scale = self.logit_scale.exp()
logits_per_image = logit_scale * image_features @ text_features.t()
logits_per_text = logits_per_image.t()
```

## 6. Entrenamiento

El hecho de que los dos _encoders_ del modelo generen vectores con las mismas dimensiones no quiere decir que puedan interpretarse como pertenecientes a un mismo espacio de características.

Un determinado valor numérico en una determinada dimensión de un vector de características generado para un texto no tiene porque tener la misma interpretación _semántica_ que el mismo valor en la misma dimensión de un vector de características generado para una imagen.

Es el entrenamiento del modelo el que garantiza que los vectores generados por el modelo sean los más similares posible para textos e imágenes relacionados, y menos similares para textos e imágenes no relacionadas.

En el repositorio del modelo no se proporciona el código de entrenamiento, pero en el _paper_ hay pseudo-código.

```python
labels = np.arange(n)

loss_i = cross_entropy_loss(logits, labels, axis=0)
loss_t = cross_entropy_loss(logits, labels, axis=1)

loss = (loss_i + loss_t)/2
```

### 6.1. Cross Entropy Loss

Para el entrenamiento se utiliza la función de pérdida de entropía cruzada (_Cross Entropy Loss_) para $N$ muestras y $C$ clases posibles.

- $ L = -\cfrac{1}{N} \sum\limits_{i=1}^{N} \sum\limits_{j=1}^{C} y_{i,j} ln\left(\cfrac{e^{x_{i,j}}}{\sum\limits_{k=1}^{C} e^{x_{i,k}}}\right) $

Esta función primero aplica la función $softmax$ en una de las dimensiones para convertir las puntuaciones brutas (_logits_) $x$ obtenidas para cada muestra en una distribución de probabilidad.

- $ \operatorname{softmax}(x_i)_j = \cfrac{e^{x_{i,j}}}{\sum\limits_{k=1}^{C} e^{x_{i,k}}} $

Y luego calcula el logaritmo con signo negativo para penalizar los errores.

- $ -\ln(\operatorname{softmax}(x_i)_j) $

Como la salida de la función $softmax$ son valores entre $0$ y $1$, donde un valor cercano a $0$ significa que la predicción es incorrecta, y cercano a $1$ que es correcta, el logaritmo con signo negativo obtiene un valor de pérdida bajo cuando la probabilidad es alta, y alto cuando la probabilidad es baja.

- $ -\ln(0) = \infty $

- $ -\ln(1) = 0 $

La pérdida para cada una de las muestra se realiza calculando el producto por las etiquetas $y$, que normalmente se expresan mediante un vector _one-hot_, donde sólo un elemento es 1 y el resto es 0, por lo que todos los productos se anulan excepto el correspondiente a la posición $j = t$ que contiene la clase correcta (_ground truth_).

- $ l_i = -y_{i,t} \ln(\operatorname{softmax}(x_i)_t) $

La pérdida final se obtiene como un único escalar calculando la media de todas las pérdidas individuales obtenidas.

- $ L = \cfrac{1}{N} \sum\limits_{i=1}^{N} l_i $

### 6.2. Aprendizaje Contrastivo

El entrenamiento se plantea como un problema de clasificación, donde los textos representan las clases en las que clasificar las imágenes, y viceversa.

El modelo se entrena con _batches_ de $N$ imágenes y $N$ textos, por lo que la salida del modelo son matrices cuadradas de _logits_ de $N \times N$ dimensiones.

La función de pérdida de entropía cruzada aplica la función $softmax$ a lo largo de una dimensión, por lo que su salida sigue siendo una matriz de $N \times N$ dimensiones.

Y, conceptualmente, el logaritmo negativo se aplica a los elementos de la diagonal, ya que para cada imagen $i$ la predicción correcta es el texto $i$, y viceversa, por lo que se obtienen $N$ pérdidas individuales.

La pérdida final se obtiene como un único escalar calculando la media de todos las pérdidas individuales obtenidas.

Este proceso se aplica dos veces, una sobre los _logits_ de las imágenes, y otra sobre los de los textos.

- $ l_I = L(S_I) $

- $ l_T = L(S_T) $

Y se calcula la pérdida total como la media de las dos pérdidas anteriores, para que el modelo aprenda a alinear los vectores de características en ambas direcciones.

- $ l = \cfrac{l_I + l_T}{2} $

El modelo aprende a generar puntuaciones brutas más elevadas sobre la diagonal.